##  Min $2x^2 + 2xy + y^2-10x-10y$ 
subject to $x^2 + y^2 ≤ 5$ 

$3x+y≤ 6$

In [23]:
import numpy as np
import ipyvolume.pylab as plt3d

X = np.linspace(-5, 5, 100)
Y = np.linspace(-5, 5, 100)

X, Y = np.meshgrid(X, Y)
Z = 2 * X**2 + 2 * X * Y + Y**2 - 10 * X - 10 * Y
H1 = -X**2 - Y**2 + 5
H2 = -3 * X - Y + 6

plt3d.figure(width=600, height=600)
plt3d.plot_surface(X, Z, Y, color='blue')
plt3d.plot_wireframe(X, H1, Y, color='yellow')
plt3d.plot_wireframe(X, H2, Y, color='green')
plt3d.ylabel("z")
plt3d.zlabel("y")
plt3d.show()

## KKT Conditions

In [3]:
import sympy
sympy.init_printing()

x, y = sympy.symbols("x, y", real=True)
f = 2 * x**2 + 2 * x * y + y**2 - 10 * x - 10 * y
h1 = -x**2 - y**2 + 5
h2 = -3 * x - y + 6

### 1.stationarity

In [4]:
f_grad = [f.diff(i) for i in (x, y)]
f_grad = sympy.Matrix(f_grad)

mu1, mu2 = sympy.symbols("mu1, mu2", real=True)
mu = sympy.Matrix([mu1, mu2])
H = sympy.Matrix([h1, h2])
H_grad = [H.diff(i) for i in (x, y)]
H_grad = sympy.Matrix(H_grad).reshape(2, 2).T

stat_cond = f_grad - sympy.Matrix(mu.T * H_grad).reshape(2, 1)
stat_cond

⎡2⋅μ₁⋅x + 3⋅μ₂ + 4⋅x + 2⋅y - 10⎤
⎢                              ⎥
⎣ 2⋅μ₁⋅y + μ₂ + 2⋅x + 2⋅y - 10 ⎦

### 2.complementary slackness

In [5]:
slack_cond = sympy.Matrix([mu1 * h1, mu2 * h2])
slack_cond

⎡   ⎛   2    2    ⎞⎤
⎢μ₁⋅⎝- x  - y  + 5⎠⎥
⎢                  ⎥
⎣μ₂⋅(-3⋅x - y + 6) ⎦

### 3.primal feasibility

In [6]:
prim_h1 = sympy.simplify(h1>=0)
prim_h2 = sympy.simplify(h2>=0)
prim_cond = sympy.Matrix([prim_h1, prim_h2])
prim_cond

⎡   2    2        ⎤
⎢- x  - y  + 5 ≥ 0⎥
⎢                 ⎥
⎣-3⋅x - y + 6 ≥ 0 ⎦

### 4.dual feasibility

In [7]:
dual_mu = sympy.simplify([mu1>=0, mu2>=0])
dual_mu = sympy.Matrix(dual_mu)
dual_mu

⎡μ₁ ≥ 0⎤
⎢      ⎥
⎣μ₂ ≥ 0⎦

### 5. Optimality

In [8]:
optimal_cond = sympy.Matrix([stat_cond, slack_cond, prim_cond, dual_mu])
optimal_cond

⎡2⋅μ₁⋅x + 3⋅μ₂ + 4⋅x + 2⋅y - 10⎤
⎢                              ⎥
⎢ 2⋅μ₁⋅y + μ₂ + 2⋅x + 2⋅y - 10 ⎥
⎢                              ⎥
⎢         ⎛   2    2    ⎞      ⎥
⎢      μ₁⋅⎝- x  - y  + 5⎠      ⎥
⎢                              ⎥
⎢      μ₂⋅(-3⋅x - y + 6)       ⎥
⎢                              ⎥
⎢         2    2               ⎥
⎢      - x  - y  + 5 ≥ 0       ⎥
⎢                              ⎥
⎢       -3⋅x - y + 6 ≥ 0       ⎥
⎢                              ⎥
⎢            μ₁ ≥ 0            ⎥
⎢                              ⎥
⎣            μ₂ ≥ 0            ⎦

In [9]:
optimal_equ = sympy.Matrix([stat_cond, slack_cond])
optimal_equ

⎡2⋅μ₁⋅x + 3⋅μ₂ + 4⋅x + 2⋅y - 10⎤
⎢                              ⎥
⎢ 2⋅μ₁⋅y + μ₂ + 2⋅x + 2⋅y - 10 ⎥
⎢                              ⎥
⎢         ⎛   2    2    ⎞      ⎥
⎢      μ₁⋅⎝- x  - y  + 5⎠      ⎥
⎢                              ⎥
⎣      μ₂⋅(-3⋅x - y + 6)       ⎦

In [31]:
roots = sympy.solve(optimal_equ.subs(mu2, 0), (x, y, mu1), real=True)
roots_float = []
for root in roots:
    t = []
    for v in root:
        t.append(v.evalf())
    roots_float.append(t)
    
roots_float

### 6. Checking

In [32]:
real_roots = roots_float.copy()
for i in range(len(roots_float)):
    if "I" in str(roots_float[i]):
        real_roots.remove(roots_float[i])
        
real_roots

In [33]:
prim_h1.subs({x: real_roots[0][0], y: real_roots[0][1]})

In [34]:
dual_mu.subs({mu1: real_roots[2][2], mu2: 0})

⎡False⎤
⎢     ⎥
⎣True ⎦

In [35]:
final_root = real_roots[1]
f_min = f.subs({x: 1, y: 2})
print("The minimum: {}".format(f_min))
print("x={:.2f}, y={:.2f}, mu1={:.2f}, mu2={:.2f}".format(final_root[0], 
                                                          final_root[1], 
                                                          final_root[2], 
                                                          0))

The minimum: -20
x=1.00, y=2.00, mu1=1.00, mu2=0.00


### Visualing the minimal value

In [36]:
final_root1 = np.array(final_root).astype(float)

plt3d.figure(width=600, height=600)
plt3d.plot_surface(X, Z, Y, color='blue')
plt3d.plot_wireframe(X, H1, Y, color='yellow') # h1: -X**2 - Y**2 + 5 >= 0
plt3d.plot_wireframe(X, H2, Y, color='green')  # h2: -3 * X - Y + 6 >= 0
plt3d.scatter(np.array([final_root1[0]]), np.array([float(f_min)]), 
              np.array([final_root1[1]]), size=5, marker="diamond", color="red")
plt3d.ylabel("z")
plt3d.zlabel("y")
plt3d.show()

### Using `Scipy`

In [37]:
from scipy import optimize

In [38]:
def f0(X):
    #2 * X**2 + 2 * X * Y + Y**2 - 10 * X - 10 * Y
    return 2 * X[0]**2 + 2 * X[0] * X[1] + X[1]**2 - 10 * X[0] - 10 * X[1] 

In [39]:
def g1(X):
    #H1 = -X**2 - Y**2 + 5
    #H2 = -3 * X - Y + 6
    return -X[0]**2 - X[1]**2 + 5

In [40]:
def g2(X):
    return -3 * X[0] - X[1] + 6

In [41]:
constraint1 = dict(type='ineq', fun=g1)
constraint2 = dict(type='ineq', fun=g2)

In [42]:
result = optimize.minimize(f0, [-5, 5],
                           constraints=[constraint1, constraint2])

In [43]:
result

     fun: -20.000000055575434
     jac: array([-1.99999809, -3.99999928])
 message: 'Optimization terminated successfully.'
    nfev: 32
     nit: 8
    njev: 8
  status: 0
 success: True
       x: array([1.0000006 , 1.99999971])

In [44]:
result.x

array([1.0000006 , 1.99999971])